# Narcissism survey analysis

**MB**: The data are from <https://openpsychometrics.org/_rawdata/>, specifically, the data for the Narcissistic Personality Inventory <http://openpsychometrics.org/_rawdata/NPI.zip>.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('mode.copy_on_write', True)
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('NPI/data.csv')
df.head()

,score,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q34,Q35,Q36,Q37,Q38,Q39,Q40,elapse,gender,age
0,18,2,2,2,2,1,2,1,2,2,...,1,1,2,2,2,1,2,211,1,50
1,6,2,2,2,1,2,2,1,2,1,...,2,1,2,2,2,2,1,149,1,40
2,27,1,2,2,1,2,1,2,1,2,...,1,2,1,1,2,1,2,168,1,28
3,29,1,1,2,2,2,1,2,1,1,...,1,2,1,2,2,1,1,230,1,37
4,6,1,2,1,1,1,2,1,2,1,...,2,1,2,2,2,0,1,389,1,50


## A partial correlation

Let's say we are interested in the scores from Q10.

We know and are not interested by the fact that the results from Q1 can partially predict the results of Q10.

What we want to know is, is whether Q2 predicts Q10, *after removing any effect that can be explained by Q1*.

We first use the code from the textbook to calculate correlations - see <https://lisds.github.io/textbook/mean-slopes/Correlation.html>.

In [3]:
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.mean(x))/np.std(x)

In [4]:
# A version of the function from the textbook.

def correlation(x_vals, y_vals):
    """ Correlation by calculation
    """
    return np.mean(standard_units(x_vals) * standard_units(y_vals))

As expected, there is a negative correlation between question 1 and question 10.

In [5]:
r_q1_q10 = correlation(df['Q1'], df['Q10'])
r_q1_q10

-0.33155950650078453

Our question is - what if we *removed* this effect (of Q1) from the values of Q10?   Would Q2 be able to predict the remaining (residual) values of Q10.  You can see this approach in <https://lisds.github.io/textbook/classification/single_multiple.html#multiple-regression-in-steps>.

We first do a regression, predicting Q10 from Q1, using Statsmodels, following the recipe in the page above.  We'll use the least-squares shortcut to minimize, for simplicity (see the page above for details).

In [6]:
from scipy.stats import linregress

In [7]:
res = linregress(df['Q1'], df['Q10'])
res

LinregressResult(slope=-0.3434917219145632, intercept=1.9857334365764383, rvalue=-0.33155950650078664, pvalue=1.2288093329619586e-286, stderr=0.009218579004519889, intercept_stderr=0.013549359121875722)

Notice this is the regression corresponding to the correlation above - you can see the previous r value in the output above.   With this regression, we have the slope and intercept to get the *predicted* values for Q10 from the values of Q1.

In [8]:
q10_predicted = res.intercept + res.slope * df['Q1']

Now we can calculate the *residuals* for Q10.  The residuals are the values left over (residual) after subtracting the predictions we got from Q1.

In [9]:
q10_residuals = df['Q10'] - q10_predicted
q10_residuals

0        0.701250
1       -0.298750
2        0.357758
3        0.357758
4        0.357758
           ...   
11238   -0.298750
11239    0.701250
11240   -0.642242
11241    0.701250
11242    0.357758
Length: 11243, dtype: float64

The residuals are the values of Q10 *after removing our estimate of the effect of Q1*.   So now we can ask our question of interest - how well does Q2 predict Q10, *after removing the effect of Q1*:

In [10]:
correlation(df['Q2'], q10_residuals)

-0.07416384855394617

This is known as the *partial* correlation of Q2 with Q10, adjusting for Q1.

Of course, we could also get the correlation with `linregress`, as we did above.  In that way, we also get the slope and intercept.

In [11]:
linregress(df['Q2'], q10_residuals)

LinregressResult(slope=-0.08538298266193917, intercept=0.1523952960132645, rvalue=-0.07416384855394625, pvalue=3.4406581645118786e-15, stderr=0.010828764848373577, intercept_stderr=0.019845386480978673)